In [1]:
import sys
sys.path.insert(0, './src/yelp_prediction')

In [2]:
import polars as pl
import dataframes as df
import matplotlib.pyplot as plt

In [3]:
(df_businesses, df_exact_stars, df_photos_per_business, df_photos) = pl.collect_all(
    (
        df.q_businesses,
        df.q_exact_stars,
        df.q_photos_agg,
        df.q_photos
    )
)

In [4]:
predictions = pl.read_csv("data/predictions.csv")
display(predictions)

epoch,photo_id,prediction
i64,str,f64
1,"""YpdQBlyCe_DAKqudC4w_6A""",4.011169
1,"""bPGQ7PHIrz2kJbWWelkZMA""",3.894624
1,"""-d6v_aUCzc5wMQJehl6SCw""",3.951678
1,"""1dzJMMWd6BKCM0d1TSpIwA""",3.867826
1,"""icyFkUFPR1u4eVNauSSqxQ""",3.811337
…,…,…
10,"""0GeqiZEuAMMjzDQTGdRf7w""",3.928407
10,"""L3ynsC89cPJCCUImHrMLxQ""",3.908075
10,"""keOGuz3lOBsyCSC6a8Mr0A""",4.210037


In [ ]:
table = (
    df_photos.join(predictions, on="photo_id")
    .join(df_exact_stars, on="business_id")
    .join(df_businesses, on="business_id")
    .join(df_photos_per_business, on="business_id")
    .select(
        pl.col("epoch"),
        pl.col("photo_id"),
        pl.col("business_id"),
        pl.col("photo_count").alias("business_photo_count"),
        pl.col("label"),
        pl.col("review_count").alias("business_review_count"),
        pl.col("prediction"),
        pl.col("exact_stars").alias("actual"),
    )
    .with_columns(
        [
            (pl.col("prediction") - pl.col("actual")).alias("error"),
        ]
    )
    .with_columns(
        [
            pl.col("error").abs().alias("error_abs"),
            pl.col("error").pow(2).alias("error_pow"),
        ]
    )
)

display(table)

epoch,photo_id,business_id,label,business_photo_count,business_review_count,prediction,actual,error,error_abs,error_pow
i64,str,str,str,u32,i64,f64,f64,f64,f64,f64
1,"""YpdQBlyCe_DAKqudC4w_6A""","""MxRZHZoDVVnN7EvMAHf1EA""","""food""",27,502,4.011169,4.211132,-0.199963,0.199963,0.039985
1,"""bPGQ7PHIrz2kJbWWelkZMA""","""MxRZHZoDVVnN7EvMAHf1EA""","""food""",27,502,3.894624,4.211132,-0.316508,0.316508,0.100177
1,"""-d6v_aUCzc5wMQJehl6SCw""","""MxRZHZoDVVnN7EvMAHf1EA""","""food""",27,502,3.951678,4.211132,-0.259454,0.259454,0.067316
1,"""1dzJMMWd6BKCM0d1TSpIwA""","""MxRZHZoDVVnN7EvMAHf1EA""","""food""",27,502,3.867826,4.211132,-0.343306,0.343306,0.117859
1,"""icyFkUFPR1u4eVNauSSqxQ""","""MxRZHZoDVVnN7EvMAHf1EA""","""food""",27,502,3.811337,4.211132,-0.399795,0.399795,0.159836
…,…,…,…,…,…,…,…,…,…,…
10,"""0GeqiZEuAMMjzDQTGdRf7w""","""XFkwQH27_LtuexEECaBQfQ""","""food""",6,59,3.928407,2.419355,1.509052,1.509052,2.277238
10,"""L3ynsC89cPJCCUImHrMLxQ""","""XFkwQH27_LtuexEECaBQfQ""","""inside""",6,59,3.908075,2.419355,1.48872,1.48872,2.216288
10,"""keOGuz3lOBsyCSC6a8Mr0A""","""XFkwQH27_LtuexEECaBQfQ""","""food""",6,59,4.210037,2.419355,1.790682,1.790682,3.206543
